<a href="https://colab.research.google.com/github/pasquale90/mthesis/blob/master/Results/Preproccess_Average_Results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Import Google_drive
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import numpy as np
import pandas as pd
import os
import ast

In [ ]:
dataset = 'us8k'#'esc50'#
exp = '4.stfts'#'3.mel'##'2.flat'#1.raw#
mode = 2#1#80# 128#360#1#16#22#32

In [ ]:
if dataset == 'esc50':
  data_path='/content/gdrive/My Drive/dissertation/ESC-50-master/meta/esc50.csv'
  num_folds = 5
  class_column = 'category'
elif dataset == 'us8k':
  data_path='/content/gdrive/My Drive/dissertation/UrbanSound8K/metadata/UrbanSound8K.csv'
  num_folds = 10
  class_column = 'class'

results_path = '/content/gdrive/My Drive/dissertation/RESULTS/'+exp+'/'+dataset+'/'+str(mode)+'/'

data = pd.read_csv(data_path)
print(data.shape)

#store_sorted_class_names, in the same way that are returned from dataset_class in data.py
classes = sorted(data[class_column].unique())
num_classes = len(classes)
print('num_classes',num_classes)
print('classes',classes)
print('num_folds',num_folds)
print('results_path : ',results_path)

# ***Average Performance***

In [ ]:
#OVERAL+FOR EACH FOLD RESULTS attempt to update
best_results = []
folds, epochs = [], []
for file in os.listdir(results_path):
 if (file != 'best_results.csv' and file != 'foldclass_results.csv' and file!='class_average.csv'):
  atrs = file.split('_')
  dataexp = atrs[0]
  fold = int(atrs [2][-1])
  if fold == 0:
    fold =10
  typeres = atrs[-1].split('.csv')[0]
  csv = pd.read_csv(results_path+file)
  if typeres == 'overalF1':
    best_result = csv.iloc[(csv['micro_f1']+csv['macro_f1']).idxmax()]
    best_epoch = best_result['epochs']
    folds.append(fold)
    best_results.append(best_result)
#best_results = best_results.sort_index(inplace=True)
best_results = pd.DataFrame(best_results,index = folds)
print (best_results)
avg_precision = best_results['macro_precision'].sum()/num_folds
avg_recall = best_results['macro_recall'].sum()/num_folds
avg_macro = best_results['macro_f1'].sum()/num_folds
print('avg_precision=',avg_precision)
print('avg_recall=',avg_recall)
print('avg_macro=',avg_macro)

In [ ]:
best_results.to_csv(results_path+'best_results.csv')    

# ***Analytical Class Performance***

In [ ]:
#Average Class_results
class_metrics = ['precision','recall','f1']
average = pd.DataFrame(data=np.zeros((num_classes,len(class_metrics))),index = classes,columns=class_metrics)
foldclass_results = pd.DataFrame(index = folds,
                                 columns=pd.MultiIndex.from_product([classes,class_metrics]))

for file in os.listdir(results_path):
  if (file.split('_')[-1].split('.csv')[0] == 'classF1'):

    fold = int(file.split('_') [2][-1])
    if fold == 0:
      fold =10

    csv = pd.read_csv(results_path+file)

    #get the selected fold from best epochs
    best_epoch = best_results['epochs'].loc[fold]
    class_results = csv.loc[csv['epochs']==best_epoch]
    
    
    for Class in class_results:
      if Class != 'epochs':
        fres = ast.literal_eval(class_results[Class].iloc[0])
        for metrics, performances in fres.items():
         if metrics != 'count':
          average.at[Class,metrics] += performances

          foldclass_results.loc[fold].at[Class,metrics]=performances
          
class_average = average/num_folds
print(class_average)
print('foldclass_results :',foldclass_results.shape)

In [ ]:
class_average.to_csv(results_path+'class_average.csv')   
foldclass_results.to_csv(results_path+'foldclass_results.csv')   

# ***WEIGHTED F1***

In [ ]:
#Compute Weighted F1 method
fc = pd.read_csv(results_path+'foldclass_results.csv', index_col=[0])
#print(fc.columns)
weighted = []
folders = []
for fold in fc.index[1:]:
  fw = 0
  for clas in fc.columns:
    if len(clas.split('.'))>1 and clas.split('.')[1]=='2':
      #print(f'fold {fold} and class {clas}')
      fw+=np.float(fc.loc[fold,clas])
  fw = fw/num_classes
  #print(fw)
  weighted.append(fw)
  folders.append(fold)


avg_wf1=np.array(weighted).sum()/num_folds
weighted.append(avg_wf1)
folders.append('avg')
perfold=pd.DataFrame(weighted,index=folders)
print(perfold)

In [ ]:
perfold.to_csv(results_path+'weightedf1.csv')   